In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import copy as copy
import matplotlib as mpl
import netCDF4 as ncd
#import torchvision
import matplotlib.cm as cm
#from torch_lr_finder import LRFinder
import copy as copy
import multiprocessing as mp
from datetime import datetime
today = datetime.today()
# custom modules
np.random.seed(100)
from scipy import stats
import time as time
  # setting random seed
def corio(lat):
    return  2*(2*np.pi/(24*60*60)) * np.sin(lat*(np.pi/180))
import matplotlib.font_manager
from cuml.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold
import pandas as pd
import gc
import pickle

In [3]:
d=ncd.Dataset("/user/wx2309/code_and_data/Data/training_data_for_SF_hbl_gaps_filled.nc").variables

l0=corio(d['l'][:])
b00=d['b0'][:]
ustar0=d['ustar'][:]
h0=d['h'][:]
lat0=d['lat'][:]
heat0=d['heat'][:]
tx0=d['tx'][:] 
tx0=np.round(tx0,2)
SF0=d['SF'][:] 

ind1=np.where(np.abs(heat0)<601)[0]
ind2=np.where(tx0<1.2)[0]
ind3=np.where(h0>29)[0]
ind4=np.where(h0<301)[0]

ind5=np.intersect1d(ind1,ind2)
ind6=np.intersect1d(ind3,ind5)
ind7=np.intersect1d(ind4,ind6)

In [4]:
def preprocess_train_data(data_load):


    ind=np.arange(0,len(data_load),1)    # creates a list of indices to shuffle
    ind_shuffle=copy.deepcopy(ind)  # deep copies the indices
    np.random.shuffle(ind_shuffle)  # shuffles the array
    
    l_mean=np.mean(data_load[:,0]); 
    l_std=np.std(data_load[:,0]); 
    data_load[:,0]=(data_load[:,0]-l_mean)/l_std    #l
    
    h_mean=np.mean(data_load[:,1]); 
    h_std=np.std(data_load[:,1]); 
    data_load[:,1]=(data_load[:,1]-h_mean)/h_std    #b0
    
    t_mean=np.mean(data_load[:,2]); 
    t_std=np.std(data_load[:,2]); 
    data_load[:,2]=(data_load[:,2]-t_mean)/t_std    #u*
    
    hb_mean= np.mean(data_load[:,3]); 
    hb_std=np.std(data_load[:,3]); 
    data_load[:,3]=(data_load[:,3]-hb_mean)/(hb_std)  #w*

    stats=np.array([l_mean, l_std, h_mean, h_std, t_mean, t_std, hb_mean, hb_std])
    tr_x=data_load[ind_shuffle,0:4]

    log_gsigma = data_load[:,4:]

    # Take the logarithm of normalized Kappa(sigma)
    for j in range(log_gsigma.shape[0]):
        log_gsigma[j,:] = np.log(log_gsigma[j,:]/np.max(log_gsigma[j,:]))

    log_gsigma_mean=np.mean(log_gsigma,axis=0)
    log_gsigma_std= np.std(log_gsigma,axis=0)

    k_points=16 #np.shape(data[:,4:])[1]

    # Demean logsigma and normalize it with standard deviation
    for k in range(k_points):
        log_gsigma[:,k]=(log_gsigma[:,k]-log_gsigma_mean[k])/log_gsigma_std[k]

    tr_y=log_gsigma
    
    return tr_x,tr_y, stats, log_gsigma_mean, log_gsigma_std

In [5]:
mm1=0; mm2=16  #0; 16
data_load_main=np.zeros([len(h0[ind7]),4+mm2-mm1])
data_load_main[:,0]=l0[ind7]
data_load_main[:,1]=b00[ind7]
data_load_main[:,2]=ustar0[ind7]
data_load_main[:,3]=h0[ind7]
data_load_main[:,4:(mm2-mm1+4)]=SF0[ind7,mm1:mm2]

data_load3=copy.deepcopy(data_load_main)

tr_x,tr_y, stats, log_gsigma_mean, log_gsigma_std=preprocess_train_data(data_load3)

valid_data=np.loadtxt('/user/wx2309/code_and_data/Data/data_testing_4_paper.txt')[:,3:]

In [6]:
ind3=np.where(valid_data[:,3]>29)[0]
ind4=np.where(valid_data[:,3]<301)[0]
ind=np.intersect1d(ind3,ind4)

valid_x=valid_data[ind,0:4]

valid_x[:,0]=(valid_x[:,0]-stats[0])/stats[1]
valid_x[:,1]=(valid_x[:,1]-stats[2])/stats[3]
valid_x[:,2]=(valid_x[:,2]-stats[4])/stats[5]
valid_x[:,3]=(valid_x[:,3]-stats[6])/stats[7]
valid_y=valid_data[ind,5:]

for i in range(len(valid_y)):
    valid_y[i,:]=np.log(valid_y[i,:]/np.max(valid_y[i,:]))

for i in range(16):
    valid_y[:,i]=(valid_y[:,i]-log_gsigma_mean[i])/log_gsigma_mean[i]

# It is found that it's computationally infeasible to build the model without GPU acceleration

# Hyperparameter tuning

In [7]:
# Randomly select 10% of data for hyper-parameter tuning
ind_tune = np.arange(0,tr_x.shape[0],1)
np.random.shuffle(ind_tune) 
num_selected = round(len(ind_tune)*0.1)
tr_x_tune = tr_x[ind_tune,:][:num_selected,:]
tr_y_tune = tr_y[ind_tune,:][:num_selected,:]

In [8]:
# Look at the default setting
scores_list = []
for i in range(16):
    scores = []
    kf = KFold()
    for tr_ind, te_ind in kf.split(np.arange(tr_x_tune.shape[0])):
        gc.collect()

        tr_x_tune_1 = tr_x_tune[tr_ind,:]
        tr_x_tune_2 = tr_x_tune[te_ind,:]
        tr_y_tune_1 = tr_y_tune[tr_ind,:]
        tr_y_tune_2 = tr_y_tune[te_ind,:]
        rfr = RFR(random_state = 66,accuracy_metric="mean_ae")
        rfr.fit(tr_x_tune_1,tr_y_tune_1[:,i])
        scores.append(rfr.score(tr_x_tune_2,tr_y_tune_2))

        del rfr
    scores_list.append(np.mean(scores))  
    
print(f"the mean absolute error over 16 nodes are {scores_list}")

/user/wx2309/.conda/envs/climate/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/user/wx2309/.conda/envs/climate/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


the mean absolute error over 16 nodes are [0.8371911792893234, 0.8364259821442221, 0.8368531286229919, 0.8368062022859423, 0.8359707409936302, 0.8353894924303444, 0.8356293116254664, 0.8360977677501442, 0.8354647271533626, 0.835805181145432, 0.8367595898873794, 0.8374543613746326, 0.8379020805788434, 0.8376159619874285, 0.8375692191734441, 0.8374975942673535]


In [9]:
# From the hyper-tuning_for_RF, the output is 
hyper_df = pd.DataFrame( {'1 column': {'max_depth': 5, 'max_features': 2, 'n_estimators': 50}, 
                          '2 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 80}, 
                          '3 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 140}, 
                          '4 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 20}, 
                          '5 column': {'max_depth': 8, 'max_features': 2, 'n_estimators': 30}, 
                          '6 column': {'max_depth': 6, 'max_features': 3, 'n_estimators': 90}, 
                          '7 column': {'max_depth': 5, 'max_features': 2, 'n_estimators': 90}, 
                          '8 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 90}, 
                          '9 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 90}, 
                          '10 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 10}, 
                          '11 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 10}, 
                          '12 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 20}, 
                          '13 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 30}, 
                          '14 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 120}, 
                          '15 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 120}, 
                          '16 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 140}})
hyper_score_list = [0.8261802564920877, 0.7558332389563769, 0.641316259121996, 0.6665849246506932,
                    0.900231269164701, 0.8273990549184675, 0.744042427685627, 0.6937356107115706, 
                    0.6758924643351577, 0.6256811290788289, 0.5977666525426375, 0.7530039657558006, 
                    0.7810863614024693, 0.7597234201281696, 0.7315923597523083, 0.6940271663365525]

base_winning_list = [ind for ind, _ in enumerate(list(np.array(scores_list) < np.array(hyper_score_list))) if _]
print(base_winning_list)
# From the ouput, it's easy to find that default estimators outperform in 5th column

[4]


In [15]:
# So the hyper-parameters selected are as follow:
hyperparameters = pd.DataFrame({'1 column': {'max_depth': 5, 'max_features': 2, 'n_estimators': 50}, 
                          '2 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 80}, 
                          '3 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 140}, 
                          '4 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 20}, 
                          '5 column': {'max_depth': 16, 'max_features': 4, 'n_estimators': 100}, 
                          '6 column': {'max_depth': 6, 'max_features': 3, 'n_estimators': 90}, 
                          '7 column': {'max_depth': 5, 'max_features': 2, 'n_estimators': 90}, 
                          '8 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 90}, 
                          '9 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 90}, 
                          '10 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 10}, 
                          '11 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 10}, 
                          '12 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 20}, 
                          '13 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 30}, 
                          '14 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 120}, 
                          '15 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 120}, 
                          '16 column': {'max_depth': 5, 'max_features': 1, 'n_estimators': 140}})
print(hyperparameters)

              1 column  2 column  3 column  4 column  5 column  6 column  \
max_depth            5         5         5         5        16         6   
max_features         2         1         1         1         4         3   
n_estimators        50        80       140        20       100        90   

              7 column  8 column  9 column  10 column  11 column  12 column  \
max_depth            5         5         5          5          5          5   
max_features         2         1         1          1          1          1   
n_estimators        90        90        90         10         10         20   

              13 column  14 column  15 column  16 column  
max_depth             5          5          5          5  
max_features          1          1          1          1  
n_estimators         30        120        120        140  


In [17]:
max_depth_list = [int(num) for num in hyperparameters.iloc[0,:]]
max_features_list = [int(num) for num in hyperparameters.iloc[1,:]]
n_estimators_list = [int(num) for num in hyperparameters.iloc[2,:]]

true_valid_gsigma = copy.deepcopy(valid_data[ind,5:])
for i in range(len(true_valid_gsigma)):
    true_valid_gsigma[i,:]= true_valid_gsigma[i,:]/np.max(true_valid_gsigma[i,:])

In [20]:
p = np.zeros(16)
valid_y_pred = np.zeros((valid_y.shape[0],valid_y.shape[1]))

for i in range(16):
    max_depth=max_depth_list[i]
    max_features=max_features_list[i]
    n_estimators=n_estimators_list[i]
    rfr = RFR(random_state=66,max_depth=max_depth,max_features=max_features,n_estimators=n_estimators,accuracy_metric = "mean_ae")

    rfr.fit(tr_x,tr_y[:,i])
    with open(f"/scratch/wx2309/climate/RF/rfr_{i+1}", "wb") as file:
        pickle.dump(rfr,file)
    valid_y_pred[:,i] = rfr.predict(valid_x)
    
    pred_valid_gsigma =  np.exp(valid_y_pred[:,i] * log_gsigma_std[i] + log_gsigma_mean[i])
    
    asd = pred_valid_gsigma - true_valid_gsigma[:,i]
    asd1 = np.percentile(asd,5)
    asd2 = np.percentile(asd,95)
    ind_iqr1=np.where(asd>asd1)[0]
    ind_iqr2=np.where(asd<asd2)[0]
    ind_iqr=np.intersect1d(ind_iqr1,ind_iqr2)
    p[i] = np.corrcoef(pred_valid_gsigma[ind_iqr], true_valid_gsigma[ind_iqr,i])[0,1]
print(f"the correlation coefficient with absolute value is {np.mean(p)}")

the correlation coefficient with absolute value is 0.32150300719187574


In [21]:
print(p)

[-0.08134161 -0.04290756  0.38445748  0.45854659  0.10413562  0.35509016
  0.47241801  0.51047595  0.34332649 -0.16433938  0.30368587  0.54133026
  0.52774066  0.54851907  0.44351016  0.43940034]
